The majority of the data pre-processing and visualization code were adapted from Professor Harry Heorton's code
My modification is to slighly adjust the date and range to prepare it for the next signal filtering process and plotting

In [ ]:
# imports all the packages
import numpy as np
import grid_set as gs
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime as dt
from dateutil.relativedelta import relativedelta
import glob
from numpy.fft import fft, ifft
from scipy.fftpack import fftfreq
import pandas as pd
import datetime
from datetime import date

In [ ]:
#### grids
m = ccrs.SouthPolarStereo()
GN = gs.grid_set(m)
GN.load_grid('D:\\Weiyi_SWH\\NSIDC_gs_SH.npz')
GN.get_grid_mask()

m = ccrs.SouthPolarStereo()
GS = gs.grid_set(m)
GS.load_grid('D:\\Weiyi_SWH\\Polar_stereo_100km_SH.npz')
GS.get_grid_mask()

In [ ]:
#### nsidc class
# This part of the code was adapted 
class NSIDC_nt():
    def __init__(self,ppath):
        self.name = 'NSIDC_n'
        self.path = ppath
# next function will take a list of dates and return an appropriately orientated arrays
# give a 
    def get_aice(self,dates_u,verbos=False):
        # does dates_u cover one year or more
        # daily files
        dimY = 316
        dimX = 332
        d_no = np.shape(dates_u)[0]
        data =  np.empty([d_no, dimX, dimY])
        for n,d in enumerate(dates_u):
            infile = self.path+d.strftime('/%Y/')+"nt_"+d.strftime('%Y%m%d')+"_*.bin"
            flist  = glob.glob(infile)
            if len(flist) > 0:
                infile = flist[0]
            else:
                print('No data found')
                return False
            with open(infile, 'rb') as fr:
                hdr = fr.read(300)
                ice = np.fromfile(fr, dtype=np.uint8)

            ice = ice.reshape(dimX,dimY)
            ice = np.flipud(ice)
            # scale it
            data[n] = ice / 250. 
            # concentration is beween 0 to 1
        data[data>1.0] = np.nan
      
        return data

    def get_dates(self,time_start,time_end):
        # does dates_u cover one year or more
        #daily files
        dates_u = []
        d_no = (time_end-time_start).days +3
        # make sure we get the bracket points
        for dn in range(d_no):
            d = time_start+ relativedelta(days = dn - 1)
            infile = self.path+d.strftime('/%Y/')+"nt_"+d.strftime('%Y%m%d')+"_*.bin"
            flist  = glob.glob(infile)
            if len(flist) > 0:
                infile = flist[0]
                dates_u.append(d)
            #if it does append dates_u
        self.dates= dates_u
        print(self.name+' Found '+str(np.shape(dates_u)[0])+' dates')


In [ ]:
#### SWH class
class CPOM_SWH_ANTO():
    """
    forcing class for the budget
    lets the forcing load efficiently
    
    """
    def __init__(self,ppath):
        d0 = dt.datetime(2000,1,1)
        self.name = 'CPOM_SWH_ANTO'
        self.path = ppath
        self.f_nc = Dataset(self.path)
        self.m = self.f_nc.dimensions['x'].size
        self.n = self.f_nc.dimensions['y'].size
        self.time_vec = self.f_nc.variables['time'][:]
        self.dates_all = [d0+relativedelta(days=t) for t in self.time_vec]
# next function will take a list of dates and return an appropriately orientated arrays
# give a 
    def get_swh(self,dates_u,verbos=False):
        # does dates_u cover one year or more
        #daily files
        idx = [np.argwhere(np.array([d == du for d  in self.dates_all] ))[0,0] 
                                             for du in dates_u]
        data = self.f_nc.variables['CryoSat2_swh'][idx].transpose(0,2,1)
        return data

    def get_dates(self,time_start,time_end):
        # does dates_u cover one year or more
        #daily files
        dates_u = [d for d in self.dates_all 
                   if d >= time_start and d <= time_end]
        self.dates = dates_u
        print(self.name+' Found '+str(np.shape(dates_u)[0])+' dates')


In [ ]:
#### SWH class
class CPOM_SWH_ANTO():
    def __init__(self,ppath):
        d0 = dt.datetime(2000,1,1)
        self.name = 'CPOM_SWH_ANTO'
        self.path = ppath
        self.f_nc = Dataset(self.path)
        self.m = self.f_nc.dimensions['x'].size
        self.n = self.f_nc.dimensions['y'].size
        self.time_vec = self.f_nc.variables['time'][:]
        self.dates_all = [d0+relativedelta(days=t) for t in self.time_vec]
# next function will take a list of dates and return an appropriately orientated arrays
# give a
    def get_swh(self,dates_u,verbos=False,mask_ice = False):
        # does dates_u cover one year or more
        #daily files
        idx = [np.argwhere(np.array([d == du for d  in self.dates_all] ))[0,0]
                                             for du in dates_u]
        data = self.f_nc.variables['CryoSat2_swh'][idx].transpose(0,2,1)
        if mask_ice:
            mask = self.f_nc.variables['mask'][idx].transpose(0,2,1)
            data[mask==0] = np.nan
        return data



    def get_dates(self,time_start,time_end):
        # does dates_u cover one year or more
        #daily files
        dates_u = [d for d in self.dates_all
                   if d >= time_start and d <= time_end]
        self.dates = dates_u
        print(self.name+' Found '+str(np.shape(dates_u)[0])+' dates')

In [ ]:
#### for the full ice concentration data edit this file path
IC = NSIDC_nt('D:\DTdata')

In [ ]:
SWH = CPOM_SWH_ANTO('D:\Weiyi_SWH\SWH_anto_100k_no_bias_2011-2019.nc')

In [ ]:
### pick a date
date = dt.datetime(2013, 3, 26)
#### use the above objects to get data for a single day
aice = IC.get_aice([date])
swh = SWH.get_swh([date])

In [ ]:
### plots SWH and IC for a single day
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(1,2,1,projection=m)
ax.set_extent([20, 90, -90, -60], ccrs.PlateCarree())
ax.pcolormesh(GN.xpts,GN.ypts,aice[0],
              vmin =0.0,vmax = 1.0)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)


ax = fig.add_subplot(1,2,2,projection=m)
ax.set_extent([-180, -130, -90, -50], ccrs.PlateCarree())
ax.pcolormesh(GS.xpts,GS.ypts,swh[0],
              vmin =0.0,vmax = 10.0)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
plt.show()

In [ ]:
### pick a series of dates
ndays = 10
date = dt.datetime(2018, 9, 5)
date_list = [date + relativedelta(days=d) for d in range(ndays)]
#### use the above objects to get data for a group of days
aice = IC.get_aice(date_list)
swh = SWH.get_swh(date_list)

In [ ]:
### plots SWH and IC for a group of days
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(1,2,1,projection=m)

#ax.set_extent([-180, -130, -90, -50], ccrs.PlateCarree())
ax.set_extent([160, 180, -90, -50], ccrs.PlateCarree())
ax.pcolormesh(GN.xpts,GN.ypts,np.nanmean(aice,axis=0),
              vmin =0.0,vmax = 1.0)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
#[180, -130, -90, -60]

ax = fig.add_subplot(1,2,2,projection=m)
ax.set_extent([160, 180, -90, -50], ccrs.PlateCarree())
#ax.set_extent([-180, -130, -90, -50], ccrs.PlateCarree())
s=ax.pcolormesh(GS.xpts,GS.ypts,np.nanmean(swh,axis=0),
              vmin =0.0,vmax = 10.0)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
plt.colorbar(s)
plt.show()

In [ ]:
### plot rate of change in IC
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1,projection=m)
#ax.set_extent([160, 180, -90, -60], ccrs.PlateCarree())
ax.set_extent([-180, -130, -90, -60], ccrs.PlateCarree())
s = ax.pcolormesh(GN.xpts,GN.ypts,np.nanmean(np.diff(aice,axis=0),axis=0),
              vmin =-0.1,vmax = 0.1,cmap = plt.cm.RdBu)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
plt.colorbar(s)
plt.show()

In [ ]:
### plot a region mask
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1,projection=m)
ax.set_extent([160, -130, -90, -60], ccrs.PlateCarree())
s = ax.pcolormesh(GS.xpts,GS.ypts,Ross_m)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
plt.colorbar(s)

plt.show()

In [ ]:
### set a time range
ts = dt.datetime(2017,1,1)
te = dt.datetime(2019,5,5)
SWH.get_dates(ts,te)
IC.get_dates(ts,te)

In [ ]:
### ice time series for regions
ice = IC.get_aice(IC.dates)
Regions_ice = [
           {'mask':Southern_m_ice,'label':'Ice concentration'}
]

for rg in Regions_ice:
    ice_region = ice*rg['mask'][None,:,:]
    plt.plot_date(IC.dates,np.nanmean(ice_region,axis=(1,2)),
                  label = rg['label'],linestyle ='-',marker = '')
plt.legend()
plt.show()

In [ ]:
### SWH time series for the Sourtern Ocean 
swh = SWH.get_swh(SWH.dates,mask_ice=True)
Regions = [{'mask':Southern_m,'label':'Significant wave height'}]

for rg in Regions:
    swh_region = swh.copy()
    mask_region = np.ones_like(swh,dtype = bool)
    mask_region *= rg['mask'][None,:,:]
    swh_region[mask_region==False] = np.nan
    
    #swh_region = swh*rg['mask'][None,:,:]
    plt.plot_date(SWH.dates,np.nanmean(swh_region,axis=(1,2)),
                  label = rg['label'],linestyle ='-',marker = '')    
        
plt.legend()
plt.show()

In [ ]:
### SWH time series for regions
swh = SWH.get_swh(SWH.dates,mask_ice=True)
Regions = [{'mask':Ross_m,'label':'West Pacific'}]

for rg in Regions:
    #swh_region = swh*rg['mask'][None,:,:]
    swh_region = swh.copy()
    mask_region = np.ones_like(swh,dtype = bool)
    mask_region *= rg['mask'][None,:,:]
    swh_region[mask_region==False] = np.nan
    # FFT the signal
    in_vec = np.nanmean(swh_region.data,axis=(1,2))
    in_vec[np.isnan(in_vec)] = 0.0
    
    time = np.arange(0,len(in_vec),1)
    W = fftfreq(len(in_vec))
    f_signal = fft(in_vec)
    # copy the FFT results
    plt.subplot(121)
    plt.plot(time,in_vec)
    plt.xlabel('date')
    plt.title('original signal')
    plt.subplot(122)
    plt.plot(W,np.abs(f_signal))
    plt.xlabel('frequency')
    plt.title('FFT after filtering')
    plt.xlim(0,0.08)
    plt.ylim(0,250)
    plt.show()
    f_signal2 = f_signal.copy()
    # low pass and high pass differ 
    # this is based on the time period
    # if it is 3 year then use the first 2 set; if it's 2 and a half use the 2nd one
    #f_signal2[0:6] = 0.0
    #f_signal2[220:] = 0.0
    f_signal2[177:] = 0.0
    f_signal2[0:5] = 0.0
    
    #f_signal2[100:] = 0.0
    #ivec = ifft(f_signal)
    ivec = ifft(f_signal2)
    plt.plot(time,in_vec,label = rg['label'])
    plt.plot(time,ivec, label = 'seasonal cycle removed')
    plt.axhline(y=8, color='r', linestyle='-',label = 'SWH threshold' )
    #plt.plot(time,3, label = 'SWH threshold')
    plt.legend()

In [ ]:
ice = IC.get_aice(IC.dates)
Regions_ice = [{'mask':Ross_m_ice,'label':'West Pacific'}]

for rg in Regions_ice:
    IC_region = ice*rg['mask'][None,:,:]
    in_vec = np.nanmean(IC_region.data,axis=(1,2))
    in_vec[np.isnan(in_vec)] = 0.0
    time_ice = np.arange(0,len(in_vec),1)
    # FFT the signal
    W = fftfreq(len(in_vec))
    f_signal = fft(in_vec)
    # copy the FFT results
    plt.subplot(121)
    plt.plot(time_ice,in_vec)
    plt.xlabel('date')
    plt.title('original signal')
    plt.subplot(122)
    plt.plot(W,np.abs(f_signal))
    plt.xlabel('date')
    plt.title('FFT after filtering')
    plt.xlim(0,0.08)
    plt.ylim(0,250)
    plt.show()
    # high-pass & low-pass filter 
    f_signal2 = f_signal.copy()
    #f_signal2[0:9] = 0.0
    #f_signal2[220:] = 0.0
    f_signal2[177:] = 0.0
    f_signal2[0:7] = 0.0
    ivec_ice = ifft(f_signal2)
    plt.plot(time_ice,in_vec,label = rg['label'])
    plt.plot(time_ice,ivec_ice, label = 'seasonal cycle removed')
    plt.legend()

In [ ]:
## checking the date
t= np.nanmean(swh_region.data,axis=(1,2))
for i in range (0, len(t)):
    # finding date where SWH above threshold
    if t[i] >9:
        #print(t[i])
        print(i)
        
        
# finding the date where large storm event happened

d0 = date(2017, 1, 1)
d1 = date(2018, 9, 5)
delta = d1 - d0
print(delta.days)

In [ ]:
# plot out the scatter plot
a=np.nanmean(swh_region.data,axis=(1,2))
c=np.nanmean(ice_region.data,axis=(1,2))
x=[]
y=[]
b=[]
data_IC = np.diff(ivec_ice) / np.diff(time_ice)
o=data_IC
p=ivec

for i in range (0,10):
    if a[i]!=float('nan') and c[i]!=float('nan'):
        x.append(i)
        y.append(p[i])
        b.append(o[i]*3000)
        #print(a[i])


plt.plot(
    x, y, 'bo',
    x, b, 'ro'
)
plt.legend(['SWH(filtered)', 'derivative of IC'])
plt.show()